In [1]:
#import required libraries
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from scipy.stats import skew

# TRIAL WITH MOVIES DATASET

## Loading the Dataset

In [27]:
df = pd.read_csv("moviesData.csv")
headers = list(df.columns.values)
df.isnull().sum().sum()

73

In [28]:
df.head()
df.drop('Unnamed: 0',axis=1)
df.head()

,Unnamed: 0,title,title_type,genre,runtime,mpaa_rating,studio,thtr_rel_year,thtr_rel_month,thtr_rel_day,...,best_actor_win,best_actress_win,best_dir_win,top200_box,director,actor1,actor2,actor3,actor4,actor5
0,1,Filly Brown,Feature Film,Drama,80.0,R,Indomina Media Inc.,2013,4,19,...,no,no,no,no,Michael D. Olmos,Gina Rodriguez,Jenni Rivera,Lou Diamond Phillips,Emilio Rivera,Joseph Julian Soria
1,2,The Dish,Feature Film,Drama,101.0,PG-13,Warner Bros. Pictures,2001,3,14,...,no,no,no,no,Rob Sitch,Sam Neill,Kevin Harrington,Patrick Warburton,Tom Long,Genevieve Mooy
2,3,Waiting for Guffman,Feature Film,Comedy,84.0,R,Sony Pictures Classics,1996,8,21,...,no,no,no,no,Christopher Guest,Christopher Guest,Catherine O'Hara,Parker Posey,Eugene Levy,Bob Balaban
3,4,The Age of Innocence,Feature Film,Drama,139.0,PG,Columbia Pictures,1993,10,1,...,yes,no,yes,no,Martin Scorsese,Daniel Day-Lewis,Michelle Pfeiffer,Winona Ryder,Richard E. Grant,Alec McCowen
4,5,Malevolence,Feature Film,Horror,90.0,R,Anchor Bay Entertainment,2004,9,10,...,no,no,no,no,Stevan Mena,Samantha Dark,R. Brandon Johnson,Brandon Johnson,Heather Magee,Richard Glover


## Initializing arrays to store the column headers for numerical and non-numerical columns of the dataset

In [29]:
numdf_col = []
strdf_col = []

## Code for dividing the dataset into numerical and non-numerical columns

In [30]:
for col in headers:
    countNum = 0
    countStr = 0
    threshold = 0.70*len(df[col])     # A threshold to qualify a column as numerical or non-numerical
    
    for item in df[col]:
        try:
            int(item)
            countNum+=1
        except ValueError:
            countStr+=1
    if(countNum>threshold):
        
        numdf_col.append(col)
    elif(countStr>threshold):
        
        strdf_col.append(col)
    else:
        continue
        

## Checking is any columns are left out

In [31]:
if(len(numdf_col)+len(strdf_col) == df.shape[1]):
    print("No columns were left out!")

No columns were left out!


In [32]:
##uncomment this block and run to see the output
#numdf_col

In [33]:
numdf = pd.DataFrame(df[numdf_col],columns=numdf_col)
strdf = pd.DataFrame(df[strdf_col],columns=strdf_col)

In [34]:
##uncomment this block and run to see the output
# strdf.head()

## Handling Unexpected values in numerical columns

In [35]:
for col in numdf_col:
    for i in range(len(numdf[col])):
        try:
            int(numdf.at[i,col])
        except ValueError:
            numdf.at[i,col]=np.nan
    

In [36]:
##uncomment this block and run to see the output
#numdf.head()

## Handling Unexpected values in non numerical columns

In [37]:
for col in strdf_col:
    for i in range(len(strdf[col])):
        try:
            int(strdf.at[i,col])
            
        except ValueError:
            continue
        else:
            strdf.at[i,col] = np.nan

In [38]:
##uncomment this block and run to see the output
#strdf.head()

## Filling numerical missing values

In [39]:
for col in numdf_col:
    skewness = skew(numdf[col])
    if(skewness>-0.5 and skewness<0.5):
        mean = numdf[col].mean()
        numdf[col].fillna(mean,inplace=True)
    else:
        median = numdf[col].median()
        numdf[col].fillna(median,inplace=True)

## Filling non-numerical values


In [40]:
imputer = SimpleImputer(missing_values=np.nan,strategy = 'most_frequent')
strdf = pd.DataFrame(imputer.fit_transform(strdf),columns=strdf_col)

## Joining the two individual dataframes into one

In [41]:
master_list = numdf_col
master_list.extend(strdf_col)
newData = pd.concat([numdf,strdf],axis=1)

In [42]:
##uncomment this block and run to see the output
#newData.head()

## Finding statistics about row_wise missing values

In [43]:
missing_row_wise = pd.DataFrame()
columns = pd.Series(newData.isnull().sum(axis=1).index.tolist())
missing_values = pd.Series(newData.isnull().sum(axis=1).tolist())
missing_row_wise = pd.concat([columns,missing_values],axis = 1, keys=["Row_Number","Missing_Count"])
#missing_row_wise

## Droping row if a large number of values are missing

In [44]:
threshold = 0.50*len(missing_row_wise['Missing_Count'])
for i in range(missing_row_wise['Row_Number'].shape[0]):
    if(missing_row_wise.at[i,'Missing_Count']>threshold):
        df.drop(df.index[i],inplace=True,axis=0)

In [45]:
newData.head()

,Unnamed: 0,runtime,thtr_rel_year,thtr_rel_month,thtr_rel_day,dvd_rel_year,dvd_rel_month,dvd_rel_day,imdb_rating,imdb_num_votes,...,best_actor_win,best_actress_win,best_dir_win,top200_box,director,actor1,actor2,actor3,actor4,actor5
0,1,80.0,2013,4,19,2013.0,7.0,30.0,5.5,899,...,no,no,no,no,Michael D. Olmos,Gina Rodriguez,Jenni Rivera,Lou Diamond Phillips,Emilio Rivera,Joseph Julian Soria
1,2,101.0,2001,3,14,2001.0,8.0,28.0,7.3,12285,...,no,no,no,no,Rob Sitch,Sam Neill,Kevin Harrington,Patrick Warburton,Tom Long,Genevieve Mooy
2,3,84.0,1996,8,21,2001.0,8.0,21.0,7.6,22381,...,no,no,no,no,Christopher Guest,Christopher Guest,Catherine O'Hara,Parker Posey,Eugene Levy,Bob Balaban
3,4,139.0,1993,10,1,2001.0,11.0,6.0,7.2,35096,...,yes,no,yes,no,Martin Scorsese,Daniel Day-Lewis,Michelle Pfeiffer,Winona Ryder,Richard E. Grant,Alec McCowen
4,5,90.0,2004,9,10,2005.0,4.0,19.0,5.1,2386,...,no,no,no,no,Stevan Mena,Samantha Dark,R. Brandon Johnson,Brandon Johnson,Heather Magee,Richard Glover


In [46]:
newData.isnull().sum().sum()

0

## Initial null/error value = 73
## Final null/error value = 0

## Optional step : To make the order of columns of the new dataframe same as that of the original dataset

In [51]:
newData = newData[headers]   #headers is the list of the columns of original dataset

In [53]:
#newData.head()

In [55]:
list(newData.columns) == list(df.columns)

True